In [ ]:
import math
import time

import PIL.Image
import ipywidgets as widgets
import torch
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import traitlets
from IPython.display import display
import cv2

from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot


In [ ]:
model_road = torchvision.models.resnet18(pretrained=False)
model_road.fc = torch.nn.Linear(512, 2)
model_road.load_state_dict(torch.load('best_steering_model_xy.pth'))

model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
model_collision.load_state_dict(torch.load('best_model.pth'))

model_signs = torchvision.models.alexnet(pretrained=False)
model_signs.classifier[6] = torch.nn.Linear(model_signs.classifier[6].in_features, 3)
model_signs.load_state_dict(torch.load('sign_detection_model.pth'))

model_kreuzung = torchvision.models.alexnet(pretrained=False)
model_kreuzung.classifier[6] = torch.nn.Linear(model_kreuzung.classifier[6].in_features, 2)
model_kreuzung.load_state_dict(torch.load('kreuzung_model.pth'))

In [1]:
device = torch.device('cuda')
model_road = model_road.to(device)
model_collision = model_collision.to(device)
model_signs = model_signs.to(device)
model_kreuzung = model_kreuzung.to(device)

NameError: name 'torch' is not defined

In [ ]:
mean = torch.Tensor([0.485, 0.456, 0.406]).half().float().to(device)
std = torch.Tensor([0.229, 0.224, 0.225]).half().float().to(device)


def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess_2(image):
    global device, normalize
    x_2 = image
    x_2 = cv2.cvtColor(x_2, cv2.COLOR_BGR2RGB)
    x_2 = x_2.transpose((2, 0, 1))
    x_2 = torch.from_numpy(x_2).float()
    x_2 = normalize(x_2)
    x_2 = x_2.to(device)
    x_2 = x_2[None, ...]
    return x_2


In [ ]:
camera = Camera.instance(width=224, height=224, fps=10)

In [ ]:
image_widget = widgets.Image()
image_vis_widget = widgets.Image()

speed_output = widgets.Text(
    value='',
    placeholder='',
    description='(l,r):',
    disabled=True
)

model_output = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Model:',
    disabled=False
)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [ ]:
# Road Following sliders
speed_control_slider = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_kd_gain_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_kd_gain_slider, steering_bias_slider)

#Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider = widgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop')
blocked_threshold = widgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

sign_verbot_slider = widgets.FloatSlider(description='signVERBOT', min=0.0, max=1.0, orientation='vertical')
sign_limit_slider = widgets.FloatSlider(description='signLIMIT', min=0.0, max=1.0, orientation='vertical')
sign_stop_slider = widgets.FloatSlider(description='signSTOP', min=0.0, max=1.0, orientation='vertical')
kreuzung_slider = widgets.FloatSlider(description='Kreuzung', min=0.0, max=1.0, orientation='vertical')

toggle_exec_btn = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    button_style='success',
    tooltip='Description',
    icon='check'
)
display(toggle_exec_btn)

display(widgets.HBox([image_widget, image_vis_widget]))
display(speed_output)

display(widgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))
display(widgets.HBox([sign_stop_slider, sign_limit_slider, sign_verbot_slider, kreuzung_slider]))

In [ ]:
speed_control, steer_gain, steer_kd_gain, steer_bias = speed_control_slider.value, 0.0, 0.0, 0.0

In [ ]:
#robot = CustomRobot()

In [ ]:
a_slider = widgets.FloatSlider(min=-math.pi, max=math.pi, step=0.01, description='a')
xy_out = widgets.Text()
display(a_slider, xy_out)

In [ ]:
def calculate_speed(last_a: float, x_in: float, y_in: float) -> (float, float, float):
    global a_slider
    a = math.atan2(x_in, y_in)
    a_slider.value = a
    xy_out.value = f"({x:.02f} {y:.02f})"
    pid = a * steer_gain + (a - last_a) * steer_kd_gain
    steer_val = pid + steer_bias
    left = max(min(speed_control + steer_val, 1.0), 0.0)
    right = max(min(speed_control - steer_val, 1.0), 0.0)
    return a, left, right

In [ ]:
def get_collision_chance(image) -> float:
    collision = model_collision(image)
    collision_softmax = functional.softmax(collision, dim=1).flatten()
    return float(collision_softmax[0])

def get_kreuzung_chance(image) -> float:
    kreuzung = model_kreuzung(image)
    kreuzung_softmax = functional.softmax(kreuzung, dim=1).flatten()
    return float(kreuzung_softmax[0])

In [1]:
def get_road_direction(image) -> (float, float):
    model_xy = model_road(image).detach().float().cpu().numpy().flatten()
    model_x = model_xy[0]
    model_y = (0.5 - model_xy[1]) / 2.0
    return model_x, model_y
def get_motor_ratio_L2R():
    return customRobot.left_motor.value / customRobot.right_motor.value

# Beispiel: offset ist 0.02
#            left_motor.value ist 0.15
#            right_motor.value ist 0.11
#    --> L2R ist 1.37 --> left 0.12, right 0.08
def dec_motor_speed_according_to_ratio(the_motor, l2r, offset):
    temp_speed = the_motor.value - (offset + 2 * offset * the_motor.value * l2r )
    if temp_speed < 0:
        temp_speed = 0
    return temp_speed

In [0]:
angle, angle_last, stop_counter = 0.0, 0.0, 0
stop_counter_limit = 10
x, y = 0.0, 0.0
frame_counter = 0

speed_default = 0.15
speed_prev_left = 0.15
speed_prev_right = 0.15
threshold_probability = 0.8 # at least 80% certainty required for flag
threshold_frames = 5 # at least 5 times in row 0.8 required for flag to be set

class IObserver:
    def update(self):
        pass

class IObservable:
    _observers = [] #indicating arrays with _
    def attach(self, observer):
        if not observer in self._observers:
            self._observers.append(observer)
    def detatch(self, observer):
        if observer in self._observers:
            self._observers.remove(observer)
    def notify(self):
        for observer in self._observers:
            observer.update()

class CustomRobot(Robot, IObserver):
    _ourObjects = []
    def setOurObjects(self, _ourObjects):
        self._ourObjects = _ourObjects
    def doYourThing(self):
        ###
        #    @todo STATE MODEL DECIDES ACTIONS BASED ON ourObject statuses. Call ourObject reactTo() methods in smart order. Decide if should follow the yellow line (PoI)
        ###

    def update(self, _probabilities): # ACHTUNG: setzt voraus, dass _probabilities und _ourObjects gleiche Reihenfolge haben !!!
        i = 0
        for ourObject in self._ourObjects:
            ourObject.setPresence(_probabilities[i])
            i+=1
        self.doYourThing()

class Scout(IObservable):
    _models = []
    device = torch.device('cuda')
    def __init__(self, _models, customRobot):
        self._models = _models
        self.attach(customRobot)

    def createUIelements(self):

    def takeNewFrame(self):

    def feedImageToModels(self):



class OurObject:
    frames_saw_in_row = 0
    frames_not_saw_in_row = 0 # Anzahl aufeinander folgender Frames in der ein is_present Objekt nicht mehr sichtbar ist
    is_present = False
    was_present = False

    # Beispiel: threshold_frames ist 4, threshold_probability ist 0.8
    # --> wenn 4 Frames hintereinander Stop-Schild zu 80% gesehen wurde, obj_sign_Stop.is_present auf True setzen
    # ansonsten counter reseten und not_saw counter erhöhen und is_present auf False setzen wenn not_saw counter 4 Frames erreicht.
    # was_present wird erst True, wenn davor is_present galt und jetzt nicht mehr
    # was_present wird in der jeweiligen reactTo() Methode auf False zurückgesetzt, wenn die entsprechend letzte Handlung durchgeführt wird.
    def setPresence(self, probability_present): # zum Beispiel prob_Kreuzung oder prob_Verbot oder ...
        if probability_present > threshold_probability:
            self.frames_saw_in_row += 1
            self.frames_not_saw_in_row = 0
            self.is_present = self.frames_saw_in_row > threshold_frames
        else:
            self.frames_saw_in_row = 0
            if self.is_present:
                self.frames_not_saw_in_row += 1
                if self.frames_not_saw_in_row > threshold_frames:
                    self.was_present = True
                    self.is_present = False

class Kreuzung(OurObject):
    def reactTo(self):
        if self.is_present:
            customRobot.set_motor_speeds(0.11, 0.11)
        else:
            customRobot.set_motor_speeds(0.15, 0.15)
            self.was_present = False

class SignVerbot(OurObject):
    phase = 0

    def __init__(self, objKreuzung):
        self.objKreuzung = objKreuzung

    def tryRight(self):
        customRobot.right(0.3)
        time.sleep(0.5)
        self.phase = 1

    def tryLeft(self):
        customRobot.left(0.3)
        time.sleep(1)
        self.phase = 2

    def goBackWhereYouCameFrom(self):
        customRobot.left(0.3)
        time.sleep(0.5)
        customRobot.set_motor_speeds(0.11, 0.11)
        self.phase = 0
        self.was_present = False

    def driveInCurrentDirection(self):
        customRobot.set_motor_speeds(0.11, 0.11)
        self.phase = 0
        self.was_present = False

    def reactTo(self):
        if self.phase == 0:
            if self.is_present:
                self.tryRight()
        elif self.phase == 1:
            if not self.objKreuzung.is_present:
                self.tryLeft()
            else:
                self.driveInCurrentDirection()
        elif self.phase == 2:
            if not self.objKreuzung.is_present:
                self.goBackWhereYouCameFrom()
            else:
                self.driveInCurrentDirection()

class SignLimit(OurObject):
    def __init__(self, objKreuzung):
        self.objKreuzung = objKreuzung

    def reactTo(self):
        if self.is_present:
            customRobot.set_motor_speeds(0.12, 0.12)
        elif self.objKreuzung.is_present:
            self.was_present = False

class SignStop(OurObject):
    def reactTo(self):
        if self.was_present:
            customRobot.stop()
            time.sleep(3)
            customRobot.set_motor_speeds(0.11, 0.11)
            self.was_present = False

class Obstacle(OurObject):
    phase = 0
    timer = 0
    patience = 5 #seconds

    def goOn(self):
        customRobot.set_motor_speeds(0.11, 0.11)
        self.was_present = False
        self.phase = 0

    def stayAndWait(self):
        customRobot.stop()
        self.timer = time.time()
        self.phase = 1

    def turnBack(self): # kurz rückwärts, 180° Wende, weiterfahren
        customRobot.backward(0.3)
        time.sleep(0.5) #@todo timer Rechnungen statt sleep
        customRobot.stop()
        customRobot.left(0.3)
        time.sleep(1)
        customRobot.stop()
        customRobot.set_motor_speeds(0.11, 0.11)
        self.phase = 0
        self.was_present = False

    def reactTo(self):
        if self.is_present:
            if self.phase == 0:
                self.stayAndWait()
            elif self.phase == 1:
                if not self.is_present:
                    self.goOn()
                elif time.time() - self.timer > self.patience:
                    self.turnBack() # @todo, soll nur 180° Wende machen, wenn es links und rechts keinen Ausweg gibt


In [3]:

signStop = SignStop()
kreuzung = Kreuzung()
signLimit = SignLimit(kreuzung)
signVerbot = SignVerbot(kreuzung)
obstacle = Obstacle()

customRobot = CustomRobot()
customRobot.setOurObjects([obstacle, signVerbot, signStop, kreuzung, signLimit])
#scout = Scout(#liste-der-modelle#, customRobot)


IndentationError: expected an indented block (2278962894.py, line 72)

In [ ]:

def execute(change):
    global angle, angle_last, blocked_slider, customRobot, stop_counter, stop_counter_limit, x, y, blocked_threshold
    global speed_control, steer_gain, steer_kd_gain, steer_bias, frame_counter
    global speed_default, speed_prev_right, speed_prev_left, threshold_probability, saw_kreuzung,saw_sign_stop, saw_sign_limit, saw_sign_verbot, is_blocked, num_turns_verbot, num_blocks, num_rueckwaerts, nichtmehr_kreuzung, nichtmehr_Stop, nichtmehr_Limit, nichtmehr_Verbot, nichtmehr_blocked
    global image, image_preproc, x_2, y_2, steer_gain, steer_kd_gain, steer_bias, stop_counter_limit, max_speed, prob_blocked, prob_sign_verbot, prob_sign_limit, prob_sign_stop, prob_kreuzung

    image = change['new']
    image_preproc = preprocess(image).to(device)
    x_2 = image
    x_2 = preprocess_2(x_2)
    y_2 = model_signs(x_2)
    y_2 = functional.softmax(y_2, dim=1)

    #@todo NICHT ALLES IN JEDEM FRAME
    #@todo kreuzung model (Fotos machen, trainieren --> kreuzung_model.pth)

    # get slider values
    steer_gain = steering_gain_slider.value
    steer_kd_gain = steering_kd_gain_slider.value
    steer_bias = steering_bias_slider.value
    stop_counter_limit = stopduration_slider.value
    max_speed = speed_control_slider.value

    prob_blocked = get_collision_chance(image_preproc)
    blocked_slider.value = prob_blocked

    prob_sign_verbot = float(y_2.flatten()[0])
    prob_sign_limit = float(y_2.flatten()[1])
    prob_sign_stop = float(y_2.flatten()[2])

    sign_verbot_slider.value = prob_sign_verbot # + prob_not_blocked - 1 # result should always be 0
    sign_limit_slider.value = prob_sign_limit
    sign_stop_slider.value = prob_sign_stop

    prob_kreuzung = get_kreuzung_chance(image_preproc)
    kreuzung_slider.value = prob_kreuzung

    '''
    Verbale Beschreibung, was das Programm tun soll:

    Wenn der Roboter blockiert ist und die Flag noch nicht gesetzt wurde,
    Flag setzen und schrittweise abbremsen.
    Im Stillstand bis zu 20 Frames warten, danach Rückwärtsgang einlegen.

    Flag bei Kreuzung setzen und langsamer werden, nach Kreuzung schneller werden (unideal bei Kurven aber egal)

    Bei Speedlimits ggf. langsamer werden. Speedlimits gelten bis zur nächsten Kreuzung.

    Bei Stopschildern Flag setzen und anhalten, wenn es nicht mehr sichtbar ist, 3s warten. (erfordert, Stopschild an gute Position zu setzen)

    Bei Durchfahrtverbotsschild 90° im Uhrzeigersinn (rechts) drehen. Bei Kreuzung weiterfahren,
    ansonsten 180° gegen den Uhrzeigersinn drehen. Bei Kreuzung weiterfahren,
    ansonsten 90° gegen den Uhrzeigersinn drehen und wieder zurück fahren

    Im Falle, dass der Roboter nicht im Rückwärtsgang ist oder sich bei Durchfahrtverbot dreht oder warten muss, den Road-Following Koordinaten (PoI) folgen

    Flags werden entsprechend wieder zurückgesetzt (hoffentlich)
    '''

    #@todo Scout, CustomRobot, mehrere OurObject erzeugen und laufen lassen

    '''
    if frame_counter % camera.fps == 0:
        frame_counter = 0

        snapshot = image.copy()
        snapshot = cv2.circle(snapshot, (int(x * snapshot.width), int(y * snapshot.height)), 8, (0, 255, 0), 3)
        image_vis_widget.value = bgr8_to_jpeg(snapshot)
    frame_counter += 1
    '''

execute({'new': camera.value})

In [ ]:
def start_stop_toggle(obj):
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        camera.observe(execute, names='value')
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        camera.unobserve(execute, names='value')
        time.sleep(0.1)
        customRobot.stop()


toggle_exec_btn.observe(start_stop_toggle, 'value')

IndentationError: expected an indented block (574240453.py, line 81)

In [ ]:
def start_stop_toggle(obj):
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        camera.observe(execute, names='value')
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        camera.unobserve(execute, names='value')
        time.sleep(0.1)
        customRobot.stop()


toggle_exec_btn.observe(start_stop_toggle, 'value')